## PREPROCESSING DATA

In [1]:
from mpmath.libmp import normalize

from preprocessing import Parser, ReportPreprocessor, SrcPreprocessor
from datasets import DATASETS

from collections import defaultdict

In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
def preprocessing_data(dataset):
    parser = Parser(dataset)
    bug_reports = parser.report_parser()
    src_files = parser.src_parser()

    os.makedirs(f'outputs/{dataset.name}', exist_ok=True)

    SrcPreprocessor(src_files).preprocess_and_export(dataset.name)
    ReportPreprocessor(bug_reports).preprocess_and_export(dataset.name)

In [ ]:
for DATASET in DATASETS:
    preprocessing_data(DATASET)

## INDEXING SOURCE CODE FILES


In [3]:
dataset_name = 'tomcat'

In [4]:
import pandas as pd
import ast
import math

In [5]:
def prepare_dataframe_src_code(name):
    return pd.read_csv(f"outputs/{name}/source_code_data.csv")

In [6]:
def fix_and_fetch_src_code_infor(data_src_code):
    def extract_stemmed(column, field = 'stemmed'):
        return column.apply(ast.literal_eval).apply(lambda x: x[field])

    stem_columns = ['pos_tagged_comments']
    for col in stem_columns:
        data_src_code[col] = extract_stemmed(data_src_code[col])

    un_stem_columns = ['file_name', 'class_names', 'method_names']
    for col in un_stem_columns:
        data_src_code[col] = extract_stemmed(data_src_code[col], 'unstemmed')

    data_src_code['natural_language'] = data_src_code['pos_tagged_comments']
    data_src_code['code_entities'] = data_src_code.apply(lambda row: row['file_name'] + row['class_names']+ row['method_names'], axis=1)

    return data_src_code[['key', 'natural_language', 'code_entities', 'total_lines']]


In [7]:
def build_inverted_index(dataset, column_data_string):
    inverted_index = defaultdict(set)
    for data in dataset.iterrows():
        for content in data[1][column_data_string]:
            inverted_index[content].add(data[1]['key'])
    return inverted_index

In [8]:
df_dataset = fix_and_fetch_src_code_infor(prepare_dataframe_src_code(dataset_name))
inverted_index_natural_language_src_codes = build_inverted_index(df_dataset, 'natural_language')
inverted_index_code_entities_src_codes = build_inverted_index(df_dataset, 'code_entities')

In [9]:
def compute_ltc(documents, inverted_index, field):
    tf_idf = defaultdict(lambda: defaultdict(float))
    total_documents = len(documents)

    for data in documents.iterrows():
        doc_id = data[1].key
        field_content = data[1][field]

        content_frequency = defaultdict(int)

        if isinstance(field_content, list) and len(field_content) > 0:
            for content in field_content:
                content_frequency[content] += 1

            for content, count in content_frequency.items():
                tf = 1 + math.log(count, 10)
                df = len(inverted_index.get(content, []))
                idf = math.log(total_documents / df, 10) if df != 0 else 0

                tf_idf[doc_id][content] = tf * idf
        else:
            tf_idf[doc_id] = {}

    for doc_id in tf_idf:
        norm = math.sqrt(sum(weight ** 2 for weight in tf_idf[doc_id].values()))
        if norm > 0:
            for content in tf_idf[doc_id]:
                tf_idf[doc_id][content] /= norm

    return tf_idf


In [10]:
natural_lang_vsm_src_codes = compute_ltc(df_dataset, inverted_index_natural_language_src_codes, 'natural_language')
code_entities_vsm_src_codes = compute_ltc(df_dataset, inverted_index_code_entities_src_codes, 'code_entities')

## CALCULATE COEFFICIENT FILE SIZE

In [11]:
min_max_size_dataset = (df_dataset.total_lines.min(), df_dataset.total_lines.max())
min_max_size_dataset

(50, 138)

In [12]:
def calculate_coefficient_size(size, size_data):
    min_size, max_size = size_data
    value_standardization = (size - min_size) / (max_size - min_size)
    return 1/(1 + math.exp(-value_standardization))

In [13]:
df_dataset['coefficient_size'] = df_dataset['total_lines'].apply(lambda x: calculate_coefficient_size(x, min_max_size_dataset))

## INDEXING BUG REPORTS

In [218]:
def prepare_dataframe_bug_reports(name):
    return pd.read_csv(f"outputs/{name}/bug_reports.csv")

In [219]:
def fix_and_fetch_bug_report(report):
    report.pos_tagged_description = report.pos_tagged_description.apply(ast.literal_eval)
    report.pos_tagged_description = report.pos_tagged_description.apply(lambda x: x['unstemmed'])

    report.pos_tagged_summary = report.pos_tagged_summary.apply(ast.literal_eval)
    report.pos_tagged_summary = report.pos_tagged_summary.apply(lambda x: x['stemmed'])

    report['fixed_files'] = report['fixed_files'].apply(lambda x: [f for f in x.split() if f != '.'] if isinstance(x, str) else x)

    report['natural_language'] = report.pos_tagged_summary
    report['code_entities'] = report.pos_tagged_description

    return report[['key', 'natural_language', 'code_entities', 'report_time', 'fixed_files']]

In [251]:
def compute_lnc(query, field):
    tf_idf = defaultdict(float)

    term_freq = defaultdict(int)
    for term in query[field]:
        term_freq[term] += 1

    for term, freq in term_freq.items():
        tf = 1 + math.log10(freq)
        tf_idf[term] = tf * 1

    norm = math.sqrt(sum(weight ** 2 for weight in tf_idf.values()))
    if norm > 0:
        for term in tf_idf:
            tf_idf[term] /= norm

    return tf_idf

In [221]:
def compute_bnc(query, field):
    tf_idf = defaultdict(float)

    unique_terms = set(query[field])

    for term in unique_terms:
        tf = 1
        tf_idf[term] = tf * 1

    norm = math.sqrt(sum(weight ** 2 for weight in tf_idf.values()))
    if norm > 0:
        for term in tf_idf:
            tf_idf[term] /= norm

    return tf_idf


In [222]:
bug_report_dataset = fix_and_fetch_bug_report(prepare_dataframe_bug_reports(dataset_name))

## CALCULATE rVSM

In [19]:
def compute_indexing_score(query, vsm, compute_query, field):
    query_vec = compute_query(query, field)
    indexing_scores = {}

    for directory, doc_vec in vsm.items():
        dot_product = 0.0
        for term in query_vec:
            dot_product += query_vec[term] * doc_vec.get(term, 0.0)

        indexing_scores[directory] = dot_product * df_dataset.loc[df_dataset['key'] == directory, 'coefficient_size'].iloc[0]

    return indexing_scores

In [20]:
def compute_rank_files_indexing(query, coefficient):
    scores_indexing = defaultdict(float)
    scores_nl = compute_indexing_score(query, natural_lang_vsm_src_codes, compute_lnc,'natural_language')
    scores_ce = compute_indexing_score(query, code_entities_vsm_src_codes, compute_lnc,'code_entities')

    for score_key in scores_ce:
        scores_indexing[score_key] = coefficient * scores_nl[score_key] + (1 - coefficient)* scores_ce[score_key]

    return scores_indexing

## GET PREVIOUS BUG REPORTS

In [21]:
def get_previous_bug_fixed_report(df, current_bug_id):
    df_sorted = df.sort_values(by='report_time')
    previous_bugs = df_sorted[df_sorted['report_time'] < df_sorted[df_sorted['key'] == current_bug_id]['report_time'].iloc[0]]
    return previous_bugs

def calculate_bug_query_similarity(df, current_bug_id):
    previous_bugs = get_previous_bug_fixed_report(df, current_bug_id)
    inverted_index = build_inverted_index(previous_bugs, 'natural_language')
    pre_bug_reports_vector = compute_ltc(previous_bugs, inverted_index, 'natural_language')

    current_bug_vector = compute_lnc(df[df.key == current_bug_id].iloc[0], 'natural_language')

    similarity_scores = {}

    for bug_id, words in pre_bug_reports_vector.items():
        dot_product = 0.0
        for term in current_bug_vector:
            dot_product += current_bug_vector[term] * words.get(term, 0.0)

        similarity_scores[bug_id] = dot_product

    return similarity_scores

In [22]:
def get_related_files(df, similarity_dict):
    relevant_bugs = {bug_id: score for bug_id, score in similarity_dict.items()}

    if not relevant_bugs:
        return {}

    file_scores = {}

    for bug_id, score in relevant_bugs.items():
        fixed_files = df[df['key'] == bug_id]['fixed_files'].iloc[0]
        for fixed_file in fixed_files:
            if fixed_file in file_scores:
                file_scores[fixed_file].append(score)
            else:
                file_scores[fixed_file] = [score]

    average_file_scores = {fixed_file: sum(fixed_file_scores) / len(fixed_file_scores) for fixed_file, fixed_file_scores in file_scores.items()}

    return average_file_scores

## CO-CHANGE MATRIX

In [23]:
import networkx as nx

G = nx.Graph()

for files in bug_report_dataset['fixed_files'].dropna():
    if len(files) < 2:
        continue
    for i in range(len(files)):
        for j in range(i + 1, len(files)):
            G.add_edge(files[i], files[j])

clusters = list(nx.connected_components(G))


In [24]:
file_to_cluster = {}
for cid, cluster in enumerate(clusters):
    for f_name in cluster:
        file_to_cluster[f_name] = cid

## FLOW

In [25]:
def calculate_final_score(report, alpha, top_k):
    rvsm_score = compute_rank_files_indexing(report, 0.4)

    similarity_bug_scores = calculate_bug_query_similarity(bug_report_dataset, report.key)
    simi_previous_bugs_score = get_related_files(bug_report_dataset, similarity_bug_scores)

    final_score = rvsm_score

    for directory in simi_previous_bugs_score:
        if directory in final_score:
            final_score[directory] = (1 - alpha) * final_score[directory] + alpha * simi_previous_bugs_score[directory]

    return dict(sorted(final_score.items(), key=lambda item: item[1], reverse=True)[:top_k])

In [26]:
import copy
import numpy as np

list_prediction = {}

for idx, bug_report in bug_report_dataset.iterrows():
    scores = calculate_final_score(bug_report, 0.2, 25)

    boosted_scores = copy.deepcopy(scores)
    cluster_scores = defaultdict(list)

    for file in scores:
        c_id = file_to_cluster.get(file)
        if c_id is not None:
            cluster_scores[c_id].append(scores[file])

    cluster_avg = {cid: np.mean(scores) for cid, scores in cluster_scores.items()}

    for file_name in scores:
        c_id = file_to_cluster.get(file_name)
        if c_id is not None:
            boosted_scores[file_name] += 0.3 * cluster_avg[c_id]

    list_prediction[bug_report.key] = boosted_scores

## EVALUATE

In [34]:
def fetch_ground_truth():
    ground_truth = {}
    for report_data in bug_report_dataset.itertuples():
        ground_truth[report_data.key] = report_data.fixed_files
    return ground_truth

In [35]:
ground_truth_data = fetch_ground_truth()

In [36]:
def top_k_accuracy(predictions, ground_truth, k=1):
    correct = 0
    total_queries = len(predictions)

    for bug_id in predictions:
        temp_prediction = dict(list(predictions[bug_id].items())[:k])
        predict_top_k = temp_prediction.keys()
        true_directory = ground_truth[bug_id]

        if any(doc in true_directory for doc in predict_top_k):
            correct += 1

    return correct / total_queries if total_queries > 0 else 0

In [37]:
top1_acc = top_k_accuracy(list_prediction, ground_truth_data, k=1)
top5_acc = top_k_accuracy(list_prediction, ground_truth_data, k=5)
top10_acc = top_k_accuracy(list_prediction, ground_truth_data, k=10)
top20_acc = top_k_accuracy(list_prediction, ground_truth_data, k=20)

print(f"Top-1 Accuracy: {top1_acc:.4f}")
print(f"Top-5 Accuracy: {top5_acc:.4f}")
print(f"Top-10 Accuracy: {top10_acc:.4f}")

Top-1 Accuracy: 0.1903
Top-5 Accuracy: 0.3892
Top-10 Accuracy: 0.4972


In [133]:
first_100_predict = dict(list(list_prediction.items())[:10])
first_100_ground_truth = dict(list(ground_truth_data.items())[:10])
top1_acc = top_k_accuracy(first_100_predict, first_100_ground_truth, k=1)
top5_acc = top_k_accuracy(first_100_predict, first_100_ground_truth, k=5)
top10_acc = top_k_accuracy(first_100_predict, first_100_ground_truth, k=10)

In [134]:
print(f"Top-1 Accuracy: {top1_acc:.4f}")
print(f"Top-5 Accuracy: {top5_acc:.4f}")
print(f"Top-10 Accuracy: {top10_acc:.4f}")

Top-1 Accuracy: 0.0000
Top-5 Accuracy: 0.1000
Top-10 Accuracy: 0.3000


## CHUNKS FILE-CODE

In [135]:
import javalang
import os
import json

In [136]:
root_dir_code = 'source_files\\tomcat-7.0.51'

In [137]:
def read_java_file(filepath, root = root_dir_code):
    file_path = os.path.join(root, filepath)
    try:
        with open(file_path, 'r', encoding='utf-8', errors= 'ignore') as java_file:
            content = java_file.read()
        return content
    except Exception as e:
        print(e)
        print(filepath)
        return ""

In [138]:
def segment_java_code(java_code):
    tree = javalang.parse.parse(java_code)
    lines = java_code.splitlines()
    segments = []

    for type in tree.types:
        for member in type.body:
            if not hasattr(member, 'position') or member.position is None:
                continue

            start_line = member.position.line - 1

            open_braces = 0
            end_line = start_line
            for i in range(start_line, len(lines)):
                open_braces += lines[i].count('{')
                open_braces -= lines[i].count('}')
                if open_braces == 0 and '{' in lines[start_line]:
                    end_line = i
                    break

            code_segment = "\n".join(lines[start_line:end_line + 1])

            if isinstance(member, javalang.tree.MethodDeclaration):
                segments.append(("method", member.name, code_segment))


    return segments

In [139]:
logic_signals = [
    'if', 'else', 'switch', 'case', 'for', 'while', 'do',

    'try', 'catch', 'finally', 'throw', 'assert',
    'IllegalArgumentException', 'NullPointerException', 'RuntimeException', 'IOException', 'SQLException',

    'log', 'logger', 'System.out', 'System.err', 'printStackTrace', 'debug', 'warn', 'trace',

    'File', 'FileReader', 'BufferedReader', 'InputStream', 'OutputStream',
    'Socket', 'HttpClient', 'URLConnection', 'request', 'response',

    '==', '!=', '<=', '>=', '&&', '||',

    'null', 'isEmpty', 'isBlank', 'validate', 'check', 'require', 'assertNotNull',

    'synchronized', 'Thread', 'Runnable', 'lock', 'unlock', 'wait', 'notify', 'Future',

    'auth', 'authenticate', 'token', 'csrf', 'secure', 'session', 'encrypt', 'decrypt'
]

def has_logic_signals(code_segment):
    return any(sig in code_segment for sig in logic_signals)

In [140]:
def count_semicolons(code_segment):
    return code_segment.count(';') >= 3

In [141]:
def group_short_chunks(chunks, max_lines=40, max_group_size=3):
    grouped_chunks = []
    i = 0

    while i < len(chunks):
        current_group = []
        total_lines = 0
        count = 0

        if len(chunks[i].splitlines()) >= max_lines:
            grouped_chunks.append(chunks[i])
            i += 1
            continue

        while i < len(chunks) and count < max_group_size:
            lines = chunks[i].splitlines()
            if total_lines + len(lines) > max_lines:
                break
            current_group.append(chunks[i])
            total_lines += len(lines)
            count += 1
            i += 1

        combined = "\n\n".join(current_group)
        grouped_chunks.append(combined)

    return grouped_chunks

In [142]:
buggy_symptoms = ['fail', 'error', 'invalid', 'exception', 'unauthorized', '403', 'throw new']

def mentions_buggy_symptoms(code_segment):
    return any(sym in code_segment.lower() for sym in buggy_symptoms)

In [143]:
skip_exact_names = ['toString', 'hashCode', 'equals', 'main']
def is_safe_name(name):
    return name in skip_exact_names or name.startswith('get') or name.startswith('set')

In [144]:
def count_logic_signals(code_segment):
    return sum(code_segment.lower().count(sig.lower()) for sig in logic_signals)

In [145]:
def is_strongly_suspicious_segment(name, code_segment, min_lines=5):
    if is_safe_name(name):
        return False
    lines = code_segment.strip().splitlines()
    vote = 0

    if len(lines) >= min_lines:
        vote += 1
    if count_logic_signals(code_segment) >= 3:
        vote += 1
    if mentions_buggy_symptoms(code_segment):
        vote += 1

    return vote >= 2


In [146]:
def build_predicted_result_chunks(directories):
    dict_chunks = {}

    for directory in directories:
        java_file = read_java_file(directory)
        chunks = segment_java_code(java_file)
        filtered_chunks = [code for seg_type, name, code in chunks if is_strongly_suspicious_segment(name, code)]
        if len(filtered_chunks) == 0:
            continue
        combined_filtered_chunks = group_short_chunks(filtered_chunks)
        dict_chunks[directory] = combined_filtered_chunks

    return dict_chunks

In [147]:
def build_list_predicted_results_chunks(predictions):
    code_dict_chunks = {}

    for index in predictions:
        chunks = build_predicted_result_chunks(predictions[index].keys())
        code_dict_chunks[index] = chunks

    return code_dict_chunks

predicted_report_chunks = build_list_predicted_results_chunks(first_100_predict)

In [148]:
pd.set_option('display.max_colwidth', None)

In [152]:
df = pd.read_csv('data/Tomcat.txt', sep='\t')
bug_summary_df = df[['bug_id', 'summary']]
bug_summary_df.set_index('bug_id')

,summary
bug_id,
56012,Bug 56012 wrong import statement generation while using the extends attribute of the page directive
56029,Bug 56029 Ternary operator doesn't work as expected inside attributes in jspx pages
55943,Bug 55943 Provide a way prevent looking at the System classloader before the webapp classloaders
56025,Bug 56025 Order of invocation of method of ServerEndpointConfig.Configurator
56010,Bug 56010 JspFactory.getPageContext with JspWriter.DEFAULT_BUFFER throws IllegalArgumentException
...,...
39689,Bug 39689 Tomcat Server Side Includes (SSI) do not work with single quotes
34801,Bug 34801 PATCH: CGIServlet does not terminate child after a timeout
40012,Bug 40012 Legacy EL Support not resolving variables correctly


In [153]:
output_file = {}
for bug_id in predicted_report_chunks:
    summary = bug_summary_df[bug_summary_df.bug_id == bug_id]['summary'].to_string(index=False).strip()
    output_file[bug_id] = {'summary': summary, 'code_segments': predicted_report_chunks[bug_id]}

In [154]:
with open('output.json', 'w') as json_file:
    json.dump(output_file, json_file, indent=4)

## LLM EVALUATE

In [155]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

In [156]:
system_message = """You are a helpful AI software engineer specializing in identifying buggy code segments given a bug report. Analyze the provided bug report and the JAVA code segment to determine if the code segment is responsible for causing the bug described in the bug report. You need to understand the functionality of the code segment and the details of the bug report to determine the relevance of the code segment to the bug report. There are two possible outputs: 'yes', 'no'.
'yes': The code is responsible for the bug described in the bug report.
'no': The code is NOT responsible for the bug described in the bug report.
Provide your output in JSON format like this sample: {{"relevance": "yes"}}.
Act like a rational software engineer and provide output. Avoid emotion and extra text other than JSON."""

def prompt_engineering(report_query, code_segment):
    return f"""Analyze the following bug report and code segment:
    Bug Report: {report_query}
    Code Segment: {code_segment}
    Please determine if the code segment is responsible for the bug described in the bug report."""

In [157]:
import re

In [158]:
def chat_llm(prompt):
    completion = client.chat.completions.create(
        model="anthropic/claude-3.5-haiku:beta",
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    content = getattr(completion.choices[0], 'message', {}).get('content') if isinstance(completion.choices[0].message, dict) else completion.choices[0].message.content

    if content is None:
        print("No content in LLM response.")
        return 'no'

    # Regex check
    match = re.search(r'"relevance"\s*:\s*"(\w+)"', content)
    if match:
        relevance = match.group(1)
        print('match result:', relevance)
        return relevance

    return 'no'


In [160]:
def llm_evaluation():
    output_relevance_segments = defaultdict(list)
    for bug_id in output_file:
        for directory in output_file[bug_id]['code_segments']:
            for i in range(len(output_file[bug_id]['code_segments'][directory])):
                relevance = chat_llm(prompt_engineering(output_file[bug_id]['summary'], output_file[bug_id]['code_segments'][directory][i]))
                if relevance == 'yes':
                    output_relevance_segments[bug_id].append(output_file[bug_id]['code_segments'][directory][i])
    return output_relevance_segments

In [161]:
relevance_segments = llm_evaluation()

match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: yes
match result: yes
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: yes
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: yes
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: no
match result: yes
match result: yes
match result: yes
match result: yes
match result: yes
match result: yes
match result: yes
mat

In [162]:
with open('output_segment.json', 'w') as json_file:
    json.dump(relevance_segments, json_file, indent=4)

## QUERY EXPANSION

In [297]:
list_10_bug_report_dataset = bug_report_dataset[:10]

In [298]:
def extract_code_entities(code_string):
    entities = set()

    method_calls = re.findall(r'\b([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', code_string)
    entities.update(method_calls)

    variable_defs = re.findall(r'\b(?:String|int|boolean|Throwable)\s+([a-zA-Z_][a-zA-Z0-9_]*)', code_string)
    entities.update(variable_defs)

    param_types = re.findall(r'\b([A-Z][a-zA-Z0-9_]*)\s+[a-zA-Z_][a-zA-Z0-9_]*', code_string)
    entities.update(param_types)

    return list(sorted(entities))

In [299]:
from assets import java_keywords

In [300]:
extracted_code_entities = defaultdict(list)
for bug_id in relevance_segments:
    for code_segment_index in range(len(relevance_segments[bug_id])):
        code_entities = [code_entity for code_entity in extract_code_entities(relevance_segments[bug_id][code_segment_index]) if code_entity not in java_keywords]
        extracted_code_entities[bug_id].append(code_entities)

for bug_id in extracted_code_entities:
    merged_code_entities = []
    for list_code_entities in extracted_code_entities[bug_id]:
        merged_code_entities.extend(list_code_entities)

    extracted_code_entities[bug_id] = merged_code_entities

In [307]:
def append_code_entities(row):
    key = row['key']
    if key in extracted_code_entities:
        existing = row['code_entities']
        print(existing)
        if not existing:
            existing = []
        return existing + extracted_code_entities[key]
    return row['code_entities']

In [312]:
import inflection
import string

In [314]:
def _split_camelcase(tokens):
    # copy token
    returning_tokens = tokens[:]
    for token in tokens:
        split_tokens = re.split(fr'[{string.punctuation}]+', token)
        # if token is split into some other tokens
        if len(split_tokens) > 1:
            returning_tokens.remove(token)
            # camelcase defect for new tokens
            for st in split_tokens:
                camel_split = inflection.underscore(st).split('_')
                if len(camel_split) > 1:
                    returning_tokens.append(st)
                    returning_tokens = returning_tokens + camel_split
                else:
                    returning_tokens.append(st)
        else:
            camel_split = inflection.underscore(token).split('_')
            if len(camel_split) > 1:
                returning_tokens = returning_tokens + camel_split
    return returning_tokens

In [317]:
from assets import stop_words

In [324]:
def preprocessing_extracted_code_entities(entities):
    temp1 = _split_camelcase(entities)
    remove_stopwords = [word for word in temp1 if word not in stop_words]
    normalize_word = [w.lower() for w in remove_stopwords]
    return normalize_word

In [325]:
for bug_id in extracted_code_entities:
    extracted_code_entities[bug_id] = preprocessing_extracted_code_entities(extracted_code_entities[bug_id])

In [308]:
list_10_bug_report_dataset

,key,natural_language,code_entities,report_time,fixed_files
0,56012,"[bug, statement, gener, use, attribut, page, direct]",[],2014-01-15 07:30:17,"[java\org\apache\jasper\compiler\PageInfo.java, java\org\apache\jasper\compiler\Validator.java]"
1,56029,"[bug, ternari, oper, work, expect, attribut, jspx, page]",[],2014-01-18 12:42:47,"[java\org\apache\jasper\compiler\ELParser.java, test\org\apache\jasper\compiler\TestELParser.java]"
2,55943,"[bug, provid, way, prevent, look, system, classload, webapp, classload]",[],2014-01-01 10:02:46,[java\org\apache\catalina\loader\WebappClassLoader.java]
3,56025,"[bug, order, invoc, method, serverendpointconfig, server, endpoint, config, configur]",[],2014-01-17 08:23:30,[java\org\apache\tomcat\websocket\server\UpgradeUtil.java]
4,56010,"[bug, illegalargumentexcept, jspfactori, jsp, factori, getpagecontext, get, page, context, jspwriter, jsp, writer, buffer, illeg, argument, except]",[],2014-01-15 03:34:23,"[java\org\apache\jasper\runtime\PageContextImpl.java, test\org\apache\jasper\runtime\TestPageContextImpl.java]"
5,55996,"[bug, async, context, timeout, http, nio, connector]",[],2014-01-13 06:12:37,[java\org\apache\tomcat\util\net\SocketWrapper.java]
6,55995,"[bug, java, lang, nullpointerexcept, pointer, except, org, apach, jasper, servlet, jspcservletcontext, jsp, servlet, context, getresourc, get, resourc, jspcservletcontext, jsp, servlet, context, java]","[pointer, exception, running, jspc, shell, script, url, object, resource, mapped, specified, path, path, path, desired, resource, exception, malformedurlexception, resource, path, formed, override, url, getresource, string, path, malformedurlexception, malformedurlexception, path, path, start, getlastmodified, tldresourcepath, tldresourcepath, result, result, result, url, exception, thread, init, jspcservletcontext, jsp, servlet, context, java, malformed, url, exception, get, resource, malformed, url, exception, path, startswith, starts, malformed, url, exception, tldcache, tld, cache, java, get, last, modified, tld, resource, path, tld, resource, path, servletcontext, servlet, context, getresource, get, resource, tldresourcepath, tld, resource, path, getwebapppath, get, webapp, path, classpath, org, apache, jasper, jspc, jsp, home, java, source, head, work, tomcat, uriroot, ...]",2014-01-13 05:55:30,[java\org\apache\jasper\compiler\TldCache.java]
7,55893,"[bug, split, accesslogvalv, extract, format, logic, abstractaccesslogvalv, access, log, valv, access, log, valv]",[],2013-12-16 21:24:37,"[java\org\apache\catalina\valves\AbstractAccessLogValve.java, java\org\apache\catalina\valves\AccessLogValve.java]"
8,55978,"[bug, initi, call, onwriteposs, made, contain, use, http, upgrad, write, possibl]","[using, writelistener, io, servlet, async, api, container, makes, call, onwritepossible, spec, using, writelistener, io, servlet, http, upgrade, api, container, make, call, onwritepossible, fixing, bug, checking, call, ondataavailable, write, listener, write, possible, write, listener, write, possible, data, available]",2014-01-08 10:37:25,"[java\org\apache\coyote\AbstractProtocol.java, java\org\apache\coyote\http11\upgrade\AbstractProcessor.java, java\org\apache\coyote\http11\upgrade\AbstractServletOutputStream.java, java\org\apache\coyote\http11\upgrade\AprServletOutputStream.java, java\org\apache\coyote\http11\upgrade\BioServletOutputStream.java, java\org\apache\coyote\http11\upgrade\NioServletOutputStream.java, test\org\apache\coyote\http11\upgrade\TestUpgrade.java]"
9,55976,"[bug, broken, respons, nio, connector, tomcat]",[],2014-01-08 09:52:04,[java\org\apache\coyote\http11\Http11NioProcessor.java]


In [327]:
list_10_bug_report_dataset['code_entities'] = list_10_bug_report_dataset['key'].apply(lambda x: extracted_code_entities[x])

C:\Users\nhatm\AppData\Local\Temp\ipykernel_41584\2904531538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_10_bug_report_dataset['code_entities'] = list_10_bug_report_dataset['key'].apply(lambda x: extracted_code_entities[x])


## RERANK

In [328]:
list_10_bug_report_dataset

,key,natural_language,code_entities,report_time,fixed_files
0,56012,"[bug, statement, gener, use, attribut, page, direct]","[string, tagplugincontext, caughtname, dotag, exceptionname, generatebody, generatejavasource, getconstantattribute, gettemporaryvariablename, hasvar, isattributespecified, removeattribute, setattribute, strvar, string, tagplugincontext, dotag, generatejavasource, getconstantattribute, getscope, hasscope, iscope, isattributespecified, removeattribute, strvar, tag, plugin, context, caught, name, tag, exception, name, generate, body, generate, java, source, get, constant, attribute, get, temporary, variable, name, var, attribute, specified, remove, attribute, set, attribute, str, var, tag, plugin, context, tag, generate, java, source, get, constant, attribute, get, scope, scope, scope, attribute, specified, remove, attribute, str, var]",2014-01-15 07:30:17,"[java\org\apache\jasper\compiler\PageInfo.java, java\org\apache\jasper\compiler\Validator.java]"
1,56029,"[bug, ternari, oper, work, expect, attribut, jspx, page]","[string, tagplugincontext, dotag, generatejavasource, getconstantattribute, getscope, hasscope, iscope, isattributespecified, removeattribute, strvar, eltext, nodes, output, stringbuilder, add, append, hasnext, length, nexttoken, parseel, parsefunction, setlength, tochar, tostring, tag, plugin, context, tag, generate, java, source, get, constant, attribute, get, scope, scope, scope, attribute, specified, remove, attribute, str, var, el, text, string, builder, next, next, token, parse, el, parse, function, set, length, char, string]",2014-01-18 12:42:47,"[java\org\apache\jasper\compiler\ELParser.java, test\org\apache\jasper\compiler\TestELParser.java]"
2,55943,"[bug, provid, way, prevent, look, system, classload, webapp, classload]","[add, classloader, filenotfoundexception, filter, jars, scis, servletcontext, string, suppresswarnings, url, addall, base, configfile, emptylist, endswith, find, getattribute, getclassloader, getname, getparent, getresource, getresources, getsystemresources, hasmoreelements, hasnext, isempty, iterator, lib, load, loadservices, matcher, next, nextelement, parseconfigfile, remove, toexternalform, driver, only, skip, the, add, clearjdbcdriverregistrations, contains, deregisterdriver, getcanonicalname, getclass, getclassloader, getdrivers, hasmoreelements, nextelement, contextconfig, file, standardcontext, suppresswarnings, tomcat, addcontext, addlifecyclelistener, getabsolutepath, getservletcontext, gettomcatinstance, load, start, testwebapp, awt, appcontext, appkit, calling, classloader, classnotfoundexception, configuration, creating, documentbuilderfactory, first, google, ioexception, ignore, illegalaccessexception, illegalargumentexception, initialise, initializing, instead, invocationtargetexception, jar, jca, jarurlconnections, java, java2d, lifecycleevent, malformedurlexception, messagedigest, method, nosuchmethodexception, parserconfigurationexception, securityexception, see, set, several, string, stringtokenizer, tccl, ...]",2014-01-01 10:02:46,[java\org\apache\catalina\loader\WebappClassLoader.java]
3,56025,"[bug, order, invoc, method, serverendpointconfig, server, endpoint, config, configur]","[configurator, deploymentexception, illegalaccessexception, instantiationexception, method, pojomethodmapping, serverendpoint, serverendpointconfig, string, validate, addendpoint, aslist, build, configurator, create, decoders, encoders, equals, getannotation, getclass, getname, getstring, getuserproperties, newinstance, path, put, subprotocols, validateencoders, value, any, endpoint, ioexception, string, wssession, close, closeauthenticatedsession, gethttpsessionid, getuserprincipal, httpsessionid, isempty, isopen, registerauthenticatedsession, registersession, remove, unregisterauthenticatedsession, unregistersession, ssl, getinstance, init, onresultrunnable, sendhandler, sendresult, setting, the, this, throwable, clearhandler, execute, init, isshutdown, onresult, p

In [329]:
def calculate_final_score_after_llm_evaluate(report):
    rvsm_score = compute_rank_files_indexing(report, 0.4)

    return dict(sorted(rvsm_score.items(), key=lambda item: item[1], reverse=True))

In [330]:
list_prediction_10_files = {}

for idx, bug_report in list_10_bug_report_dataset.iterrows():
    scores = calculate_final_score_after_llm_evaluate(bug_report)

    list_prediction_10_files[bug_report.key] = scores

bug
statement
gener
use
attribut
page
direct
string
tagplugincontext
caughtname
dotag
exceptionname
generatebody
generatejavasource
getconstantattribute
gettemporaryvariablename
hasvar
isattributespecified
removeattribute
setattribute
strvar
string
tagplugincontext
dotag
generatejavasource
getconstantattribute
getscope
hasscope
iscope
isattributespecified
removeattribute
strvar
tag
plugin
context
caught
name
tag
exception
name
generate
body
generate
java
source
get
constant
attribute
get
temporary
variable
name
var
attribute
specified
remove
attribute
set
attribute
str
var
tag
plugin
context
tag
generate
java
source
get
constant
attribute
get
scope
scope
scope
attribute
specified
remove
attribute
str
var
bug
ternari
oper
work
expect
attribut
jspx
page
string
tagplugincontext
dotag
generatejavasource
getconstantattribute
getscope
hasscope
iscope
isattributespecified
removeattribute
strvar
eltext
nodes
output
stringbuilder
add
append
hasnext
length
nexttoken
parseel
parsefunction
setleng

In [331]:
accuracy_1 = top_k_accuracy(list_prediction_10_files, first_100_ground_truth, 1)
accuracy_5 = top_k_accuracy(list_prediction_10_files, first_100_ground_truth, 5)
accuracy_10 = top_k_accuracy(list_prediction_10_files, first_100_ground_truth, 10)


In [332]:
print(accuracy_1, accuracy_5, accuracy_10)

0.2 0.3 0.5


In [333]:
list_prediction_10_files

{56012: {'java\\org\\apache\\jasper\\compiler\\tagplugin\\TagPluginContext.java': 0.21867248650594495,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core\\Remove.java': 0.1564823955070831,
  'java\\org\\apache\\jasper\\compiler\\TagPluginManager.java': 0.14520618965711146,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core\\Catch.java': 0.1403355143571687,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core\\When.java': 0.11886268956451049,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core\\Import.java': 0.11544183624334171,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core\\Set.java': 0.11460045583806223,
  'java\\org\\apache\\jasper\\compiler\\TagConstants.java': 0.10547717320841757,
  'java\\javax\\servlet\\jsp\\JspContext.java': 0.09439287562802158,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core\\If.java': 0.0896375038623569,
  'java\\org\\apache\\jasper\\compiler\\tagplugin\\TagPlugin.java': 0.08739303882278637,
  'java\\org\\apache\\jasper\\tagplugins\\jstl\\core